# Creating a feature matrix from a DB Query

In [1]:
import sys
sys.path.append('/Users/shayneufeld/GitHub/mouse_bandit/data_preprocessing_code')
sys.path.append('/Users/shayneufeld/GitHub/mouse_bandit')
import support_functions as sf
import numpy as np
import pandas as pd
import scipy as sp
import bandit_preprocessing as bp
from sklearn import preprocessing
import sys
import os
%matplotlib inline 

## Retrieve names of sessions you want from the DB

In [2]:
#load in data base
db = pd.read_csv('/Users/shayneufeld/GitHub/mouse_bandit/session_record.csv',index_col=0)

### Query all 80-20 sessions where performance > 0.7 and block structure was 50

In [5]:
r = db[((db['Left Reward Prob'] == 1.0) |  (db['Right Reward Prob'] == 1.0))].copy()
r = r[r['p(high Port)'] > 0.85].copy()
r = r[r['Block Range Min'] == 50].copy()
session_names = r['Session ID'].values

In [6]:
r

,Session ID,Mouse ID,Date,Phase,Left Reward Prob,Right Reward Prob,Block Range Min,Block Range Max,No. Trials,No. Blocks,No. Rewards,p(high Port),Decision Window Duration,Min Inter-trial-interval,Left Solenoid Duration,Right Solenoid Duration
65,10312016_q43,q43,10312016,2.0,0.0,1.0,50.0,50.0,387.0,6.0,349.0,0.90,2.0,1.0,45.0,40.0
200,08012016_dumble,dumble,8012016,2.0,1.0,0.0,50.0,50.0,466.0,8.0,410.0,0.88,2.0,1.0,45.0,40.0
201,08012016_harry,harry,8012016,2.0,1.0,0.0,50.0,50.0,448.0,8.0,422.0,0.94,2.0,1.0,45.0,40.0
202,08012016_quirrel,quirrel,8012016,2.0,1.0,0.0,50.0,50.0,438.0,8.0,400.0,0.91,2.0,1.0,45.0,40.0
203,08012016_sprout,sprout,8012016,2.0,1.0,0.0,50.0,50.0,473.0,8.0,405.0,0.86,2.0,1.0,40.0,40.0
204,08012016_tom,tom,8012016,2.0,1.0,0.0,50.0,50.0,477.0,8.0,425.0,0.89,2.0,1.0,40.0,40.0
205,08012016_tonks,tonks,8012016,2.0,1.0,0.0,50.0,50.0,451.0,8.0,404.0,0.90,2.0,1.0,40.0,40.0
206,08012016_volde,volde,8012016,2.0,0.0,1.0,50.0,50.0,505.0,9.0,455.0,0.90,2.0,1.0,40.0,40.0
286,08192016_quirrel,quirrel,8192016,2.0,1.0,0.0,50.0,50.0,451.0,8.0,402.0,0.89,2.0,1.0,25.0,25.0
288,08192016_tom,tom,8192016,2.0,1.0,0.0,50.0,50.0,455.0,8.0,405.0,0.89,2.0,1.0,25.0,30.0


In [7]:
r.shape

(21, 16)

## load in csv files (from running exportTrials.m)

In [8]:
'''
load in trial data
'''
columns = ['Elapsed Time (s)','Since last trial (s)','Trial Duration (s)','Port Poked','Right Reward Prob','Left Reward Prob','Reward Given']

root_dir = '/Users/shayneufeld/GitHub/mouse_bandit/data/trial_data'

trial_df = []

for session in session_names:
    full_name = session + '_trials.csv'
    
    path_name = os.path.join(root_dir,full_name)
    
    trial_df.append(pd.read_csv(path_name,names=columns))

mouse_ids = r['Mouse ID'].values

In [9]:
len(trial_df)

21

## convert into 1 feature matrix

In [11]:
for i,df in enumerate(trial_df):
    
    curr_feature_matrix = bp.create_reduced_feature_matrix(df,mouse_ids[i],session_names[i],feature_names='Default')
    
    if i == 0:
        master_matrix = curr_feature_matrix.copy()
    else:
        master_matrix = master_matrix.append(curr_feature_matrix)
    

In [30]:
master_matrix.shape

(17856, 50)

In [31]:
master_matrix['Higher p port'].mean()

0.75128808243727596

## Save combined feature matrix  

In [32]:
master_matrix.to_csv(os.path.join(root_dir,'reduced_1000_02192017.csv'))